In [1]:
%%javascript
IPython.notebook.save_notebook()
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
# #exclude
# from ffwdpy import JupNb
# jupnb = JupNb()
# jupnb.write_nb2py(nb_name)

In [3]:
%%javascript

IPython.tab_as_tab_everywhere = function(use_tabs) {
	if (use_tabs === undefined) {
		use_tabs = true; 
	}

	// apply setting to all current CodeMirror instances
	IPython.notebook.get_cells().map(
		function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
	);
	// make sure new CodeMirror instances created in the future also use this setting
	CodeMirror.defaults.indentWithTabs=use_tabs;
	
	};

IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [4]:
# GPU setup
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="8"

# Import Necessary Packages

In [5]:
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import os
import numpy as np
import torch
from tqdm.notebook import tqdm
from torchvision import transforms
import matplotlib.pyplot as plt
import random
import numpy as np
from PIL import Image
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import sklearn
from torch import nn
from torch.nn.utils.rnn import *
import fnmatch
from torch.utils.tensorboard import SummaryWriter
import clip 

cuda = torch.cuda.is_available()
print("cuda", cuda)
num_workers = 8 if cuda else 0
print(num_workers)
print("Torch version:", torch.__version__)

cuda True
8
Torch version: 1.7.1+cu110


In [6]:
import sys
sys.path.insert(0, "/data3/puppala/projects/")

In [7]:
from Vid_CLIP.utils.labels import labels, val_labels, sub_labels
from Vid_CLIP.utils.templates import imagenet_templates as templates
from Vid_CLIP.models.transformer import Pool_Transformer_Model
from Vid_CLIP.datasets.kinetics import Mapped_Dataset, Unmapped_Dataset

SyntaxError: invalid syntax (transformer.py, line 218)

In [ ]:
classnames = labels
map_id = {}
i=0
for classname in classnames:
	map_id[classname]=i
	i+=1

In [ ]:

model, _ = clip.load("RN50") 
def zeroshot_classifier(classnames, act_descriptions):
	with torch.no_grad():
		zeroshot_weights = []
		for classname in classnames:
			texts = [template.format(classname) for template in templates]
			# pdb.set_trace()
			texts = clip.tokenize(texts).cuda() #tokenize
			class_embeddings = model.encode_text(texts) #embed with text encoder
			class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
			class_embedding = class_embeddings.mean(dim=0)
			class_embedding /= class_embedding.norm()
			zeroshot_weights.append(class_embedding)
		zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
	return zeroshot_weights
classnames_str = {x:x.replace('_', ' ') for x in classnames}
zeroshot_weights = zeroshot_classifier(classnames, classnames_str)

In [ ]:
# Dataloader train

val_dataset = Mapped_Dataset("/data3/puppala/data/kinetics_embeddings/validation")
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8)
NUM_CLASSES = len(val_dataset.class_names)
class_names = val_dataset.class_names

train_dataset = Mapped_Dataset("/data3/puppala/data/kinetics_embeddings/training" class_names = class_names)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)

batch_feats,batch_labels,batch_lengths = next(iter(train_dataloader))
print('feats - ', batch_feats.shape)
print('labels - ', batch_labels.shape)
print('lengths - ', batch_lengths.shape)

## Train and Test Loop

In [ ]:
# Set the hyperparameters of the model
numEpochs = 20
num_feats = 1024
learningRate = 1e-3
weightDecay = 5e-6
num_classes = 25


In [ ]:
# Model Initialisation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = Transformer_Model(ntoken=100, ninp=100, nhead=5, nhid=4, nlayers=2)
model = Pool_Transformer_Model(spacial_dim=10, embed_dim=1024, num_heads=8, output_dim=1024)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)
writer = SummaryWriter(log_dir='runs/transformer')
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
# Train Function
def validate(model, data_loader):
	model.eval()
	correct = 0
	total = 0
	loss = []
	for batch_num, (feats,labels,lengths) in enumerate(data_loader):
		feats,labels = feats.to(device),labels[:,0].to(device)
		out = model(feats)
		curr_loss = criterion(out, labels.long())
		correct += (torch.argmax(out,dim=1)==labels).sum().detach().cpu().numpy()
		total += feats.shape[0]
		# Compute loss
		loss.append(curr_loss.item()) 

	avg_loss = np.mean(loss)
	print("Accuracy:",correct/total)
	writer.add_scalar('Val/Loss', avg_loss) 
	writer.add_scalar('Val/Accuracy', correct/total) 
	return avg_loss,correct/total

In [ ]:
# Train Function
train_loss = []
val_loss = []
val_acc = []
def train(model, data_loader,numEpochs,val_dataloader):
	model.train()
	for epoch in range(numEpochs):
		avg_loss = 0.0    
		for batch_num, (feats,labels,lengths) in enumerate(data_loader):
			torch.autograd.set_detect_anomaly(True)
			optimizer.zero_grad()
			feats,labels = feats.to(device),labels[:,0].to(device)
			# print(feats.shape)
			out = model(feats)
			loss = criterion(out, labels.long())
			loss.backward()
			optimizer.step()         
			avg_loss += loss.item()
			if batch_num % 50 == 49:
				print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
				train_loss.append(avg_loss/50)
				writer.add_scalar('Train/Loss', avg_loss/50)
				avg_loss = 0.0  
				vloss,vacc = validate(model,val_dataloader)
				val_loss.append(vloss)
				val_acc.append(vacc)
				model.train()
		scheduler.step()

In [ ]:
train(model,train_dataloader,numEpochs,val_dataloader)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_loss)
plt.plot(val_loss)
plt.title("Train and Validation Loss")
plt.legend(['Train Loss','Val Loss'])
plt.xlabel('Number of iterations')
plt.ylabel('Loss value')

In [ ]:
plt.plot(val_acc)
plt.title("Validation Accuracy")
plt.xlabel('Number of iterations')
plt.ylabel('Accuracy')
_ = plt.yticks([0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9])